In [1]:
#Necessary Imports
!pip install ta
import numpy as np
import pandas as pd
import os
import pickle
import ta
from statsmodels.tsa.arima_model import ARIMA
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/mining-on-stock") #Data Source Location

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29103 sha256=9a38ac04acb9f44b3faa278987982a137db647553337789bc15fb8e5fbda7328
  Stored in directory: /root/.cache/pip/wheels/47/51/06/380dc516ea78621870b93ff65527c251afdfdc5fa9d7f4d248
Successfully built ta
Mounted at /content/drive


In [2]:
#Fetching training data from the pkl file.
with open('training_set.pkl', 'rb') as train_file:
  train_data = pickle.load(train_file)

#We can also use pd.read_pickle fuction

In [3]:
#Basic Stats of the training data.
print("Data Storage Type of Complete data => ", type(train_data), '\n')
print("Total Number of Records => ", len(train_data), '\n')
print("Data Type of each record => ", type(train_data[0]), '\n')
print("Total Entries in each record => ", len(train_data[0]), '\n')
print("Head of data => \n", train_data[0].head(), '\n')
print("tail of data => \n", train_data[0].tail(), '\n')
print("Head of data => \n", train_data[-1].head(), '\n')
print("tail of data => \n", train_data[-1].tail(), '\n')
print("Basic Stats about data", train_data[0].describe(), '\n')
print("Total Records => ", len(train_data) * len(train_data[0]))

Data Storage Type of Complete data =>  <class 'list'> 

Total Number of Records =>  2000 

Data Type of each record =>  <class 'pandas.core.frame.DataFrame'> 

Total Entries in each record =>  2202 

Head of data => 
        Open      High       Low     Close    Volume
0  0.672106  0.679902  0.669265  0.671966  0.336899
1  0.672106  0.674597  0.670546  0.672106  0.208023
2  0.672106  0.678621  0.671827  0.676282  0.142405
3  0.676338  0.677340  0.672384  0.673498  0.146400
4  0.673442  0.676004  0.672997  0.675892  0.116960 

tail of data => 
           Open      High       Low     Close    Volume
2197  0.680849  0.683633  0.672997  0.674055  0.351048
2198  0.674389  0.674890  0.671326  0.674055  0.143069
2199  0.674110  0.676227  0.672440  0.676004  0.312048
2200  0.675725  0.675837  0.668820  0.669321  0.279230
2201  0.669293  0.671326  0.662973  0.664198  0.552933 

Head of data => 
        Open      High       Low     Close    Volume
0  0.791198  0.809398  0.768696  0.772005  0.098

In [4]:
# Concatenate the daily percentage changes for all the stocks into a single list
change_list = [df['Close'].pct_change() for df in train_data]
all_changes = sorted([item for sublist in change_list for item in sublist])
all_changes = np.nan_to_num(all_changes)

# Compute the two thresholds that divide the list into three equal-sized intervals
level1 = np.percentile(all_changes, 33.33)
level2 = np.percentile(all_changes, 66.67)

# Assign labels to the daily percentage changes for each stock
labels = []
for df in train_data:
    close_pct_change = df['Close'].pct_change() 
    label = pd.cut(close_pct_change, bins=[-np.inf, level1, level2, np.inf], labels=['decrease', 'no big change', 'increase'])
    labels.append(label)
    
# Output the thresholds and the number of data points in each level
print(f'Threshold 1: {level1:.6f}')
print(f'Threshold 2: {level2:.6f}')
labels = pd.concat(labels)
print(f'Total number of data points: {len(labels)}')
print(f'Number of data points in decrease level: {(labels == "decrease").sum()}')
print(f'Number of data points in no big change level: {(labels == "no big change").sum()}')
print(f'Number of data points in increase level: {(labels == "increase").sum()}')

Threshold 1: -0.002106
Threshold 2: 0.001981
Total number of data points: 4404000
Number of data points in decrease level: 1467857
Number of data points in no big change level: 1466290
Number of data points in increase level: 1467853


In [ ]:
#These are the functions that we have considered using for making additional features in our dataset. 
feature_funcs = [('awesome_oscillator', None),
 ('bollinger_percent_b', None),
 ('cfo', None),
 ('cmo', None),
 ('coppock_curve', None),
 ('detrended_price_oscillator', None),
 ('disparity_index', None),
 ('elder_impulse_system', None),
 ('gator_oscillator', None), 
 ('historical_volatility', None),
 ('intraday_momentum_index', None),
 ('linear_reg_slope', None),
 ('macd', None),
 ('macd_divergence', None), 
 ('momentum_indicator', None), 
 ('qstick', 14), 
 ('projected_volume_at_time', 14),
 ('projected_aggregate_volume', 14),
 ('price_volume_trend', None),
 ('positive_volume_index', None),
 ('twiggs_money_flow', None),
 ('volume_underlay', None),
 ('on_balance_volume', None),
 ('moving_average', 30),
 ('parabolic_sar', (0.02, 0.2)),
 ('pretty_good_oscillator', (10, 20)),
 ('price_momentum_oscillator', (12, 24)),
 ('price_oscillator', (12, 26, 9)),
 ('price_rate_of_change', 14),
 ('KST', (10, 15, 20, 30, 10, 10, 10, 15)),
 ('SpecialK', (10, 15, 20, 30, 10, 10, 10, 15)),
 ('ravi', (7, 65)),
 ('rsi', 14),
 ('rsi_divergence', (14, 9)),
 ('rainbow_moving_average', 30),
 ('rainbow_oscillator', (10, 1.0)),
 ('random_walk_index', 14),
 ('relative_vigor_index', 14),
 ('schaff_trend_cycle', 10),
 ('standard_deviation', 14),
 ('stochastic_divergence', 14),
 ('stochastic_momentum_index', (14, 3)),
 ('stochastic_rsi', (14, 3, 3)),
 ('stochastics', (14, 3, 3)),
 ('trix', 15),
 ('trade_volume_index', None),
 ('trend_intensity_index', 14),
 ('true_range', None),
 ('typical_price', None),
 ('ultimate_oscillator', (7, 14, 28, 4, 2, 1)),
 ('vwap', None),
 ('vertical_horizontal_filter', 28),
 ('vortex_indicator', 14),
 ('williams_r', 14),
 ('zigzag', 5),
 ('calculate_adx', None), 
 ('atr_bands', None),
 ('accumulative_swing_index', None),
 ('alligator', None),
 ('aroon', None),
 ('aroon_oscillator', None),
 ('average_true_range', None),
 ('cog', None),
 ('cpr', None),
 ('chv', None),
 ('cmf', None),
 ('ci', None),
 ('cci', None),
 ('donchian_channel', None),
 ('ehlers_fisher_transform', None),
 ('elder_ray_index', None),
 ('fractal_chaos_bands', None),
 ('fractal_chaos_oscillator', None),
 ('gopalakrishnan_range_index', None),
 ('high_minus_low', None),
 ('highest_high', None),
 ('ichimoku_clouds', None),
 ('keltner_channel', None),
 ('linear_reg_forecast', None),
 ('mass_index', None),
 ('median_price', None),
 ('money_flow_index', None)]
 #There are about 82 functions above from which we have obtained more than 100 new features.
 #With careful evaluation, we finally considered using 101 features for task-3 removing all other erratic features.

In [ ]:
len(feature_funcs)

82

In [ ]:
# Define a function to calculate the new features
from features import *
# print(dir(features))
def calculate_new_features(df):
    df_new = df.copy()
    for func in feature_funcs:
      func_name = func[0]
      func_args = func[1]
      # Import the function dynamically using its name
      func_module = __import__('features', globals(), locals(), [func_name], 0)
      func_to_call = getattr(func_module, func_name)
      # print(dir(func_module))
      # print(func_name, func_args)
      # Call the function with its arguments
      try:
        if func_args is None: 
            result = func_to_call(df)
        elif isinstance(func_args, tuple):
            result = func_to_call(df, *func_args)
        else:
            result = func_to_call(df, func_args)   
        if isinstance(result, pd.DataFrame):
          for i in result.columns.tolist():
            column_name = f'{func_name} ({i})'
            df_new[column_name] = result[i]
        elif isinstance(result, pd.Series):
            column_name = f'{func_name} ({result.name})'
            df_new[column_name] = result
      except AttributeError:
          # handle the exception
          print(f"{func_name} function not found in the features module")
      except Exception as e:
          # handle any other exception
          print(f"An error occurred while calling {func_name}: {str(e)}")
    return df_new

# Create a new list to store the processed dataframes
processed_data = []

# Iterate over each dataframe in the list
for df in train_data.copy():
    # Apply the calculate_new_features function to the current dataframe
    df_new = calculate_new_features(df)
    # Append the updated dataframe to the new list
    processed_data.append(df_new)
#print(processed_data)

In aroon_oscillator function


<ipython-input-8-518118f18b88>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result[i]
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

In aroon_oscillator function


<ipython-input-8-518118f18b88>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result[i]
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

In aroon_oscillator function


...

<ipython-input-8-518118f18b88>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result[i]
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

In aroon_oscillator function


<ipython-input-8-518118f18b88>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result[i]
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

In aroon_oscillator function


<ipython-input-8-518118f18b88>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result[i]
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[column_name] = result
<ipython-input-8-518118f18b88>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

In [ ]:
with open('data_with_new_features.pkl', 'wb') as f:
    pickle.dump(processed_data, f)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score
import joblib

In [ ]:
# Separate the features and the labels
temp_data = train_data.drop(['vwap (None)'], axis=1)
temp_data = temp_data.dropna(subset=['Label'])

X = temp_data.iloc[:, :-1]
y = temp_data.iloc[:, -1]

In [ ]:
print(len(X))
len(y)

1100500


1100500

In [ ]:
print(y)

1       no big change
2            increase
3            decrease
4            increase
5       no big change
            ...      
2197         decrease
2198         increase
2199    no big change
2200         decrease
2201         decrease
Name: Label, Length: 1100500, dtype: category
Categories (3, object): ['decrease' < 'no big change' < 'increase']


In [ ]:
one_hot_encoded = pd.get_dummies(temp_data["Label"])
y_ = np.array(one_hot_encoded)
y_encoded = list(y_)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# create OneHotEncoder object
enc = OneHotEncoder()

# fit and transform the target variable
y_encoded = enc.fit_transform(y.values.reshape(-1, 1))

# convert sparse matrix to array
y_encoded = y_encoded.toarray()


(3,)

In [ ]:
enc = OneHotEncoder(sparse=False)
y_encoded = enc.fit_transform(y.values.reshape(-1, 1))

NameError: ignored

In [ ]:
y_encoded[0].shape

array([0., 0., 1.])

In [ ]:
'''from sklearn.preprocessing import LabelEncoder

# encode the target variable y using label encoding
le = LabelEncoder()
y_encoded = le.fit_transform(y)'''

In [ ]:
'''from sklearn.preprocessing import OneHotEncoder
# apply one-hot encoding to the label
encoder = OneHotEncoder()
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1)).toarray()'''

In [ ]:
# Check for NaN values
print(X.isnull().sum())

X = X.fillna(X.mean())
print(X.isnull().sum())

Open                                      0
High                                      0
Low                                       0
Close                                     0
Volume                                    0
                                       ... 
keltner_channel (Keltner Lower)        9000
linear_reg_forecast (LRF)                 0
mass_index (Mass_Index)                7500
median_price (Median Price)               0
money_flow_index (Money Flow Index)    6508
Length: 108, dtype: int64
Open                                   0
High                                   0
Low                                    0
Close                                  0
Volume                                 0
                                      ..
keltner_channel (Keltner Lower)        0
linear_reg_forecast (LRF)              0
mass_index (Mass_Index)                0
median_price (Median Price)            0
money_flow_index (Money Flow Index)    0
Length: 108, dtype: int64


In [ ]:
X[np.isfinite(X) == False] = np.finfo('float64').max
X = np.clip(X, a_min=None, a_max=1e10)
# # Split data into training set and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
print(len(X_train), len(X_val), len(y_train), len(y_val))

X_train = X_train.dropna()
print(len(X_train), len(X_val), len(y_train), len(y_val))

880400 220100 880400 220100
880400 220100 880400 220100


In [ ]:
# define the hyperparameter grid
param_grid = {'kernel': ['linear', 'rbf'], 'C': [1, 10, 100], 'gamma': ['scale', 'auto']}

# define the scoring metrics and refit the model using precision
scoring = ['accuracy', 'precision']
refit_score = 'precision'

# create the SVM model
svm_model = SVC()

# define the GridSearchCV object
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, scoring=scoring, refit=refit_score, cv=5, error_score='raise')
grid_search.fit(X_train, y_train)

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score

# split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.5, random_state=42)

# define the model with optimized hyperparameters
hist_model = HistGradientBoostingClassifier(
    max_depth=10,
    max_iter=200,
    learning_rate=0.1,
    min_samples_leaf=50,
    max_leaf_nodes=5000,
    validation_fraction=0.1,
    n_iter_no_change=10,
    verbose=1,
    random_state=42
)

# define the scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score), 'precision': make_scorer(precision_score, average='weighted')}

# compute the cross-validation scores
#cv_results = cross_validate(hist_model, X, y_encoded, cv=5, scoring=scoring)

# train the model on the training set
hist_model.fit(X_train, y_train)

# predict on the training set and compute the accuracy and precision scores
y_train_pred = hist_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='weighted')

# predict on the validation set and compute the accuracy and precision scores
y_val_pred = hist_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_precision = precision_score(y_val, y_val_pred, avergae='weighted')

# calculate the percentage of positive predictions
train_pct_pos = sum(y_train_pred == np.array([0, 0, 1])) / len(y_train_pred) * 100
val_pct_pos = sum(y_val_pred == np.array([0, 0, 1])) / len(y_val_pred) * 100

# print the results
print("Training set accuracy:", train_accuracy)
print("Training set precision:", train_precision)
print("Validation set accuracy:", val_accuracy)
print("Validation set precision:", val_precision)
print("Percentage of positive predictions on training set:", train_pct_pos)
print("Percentage of positive predictions on validation set:", val_pct_pos)
print("Mean accuracy score from cross-validation:", cv_results['test_accuracy'].mean())
print("Standard deviation of accuracy score from cross-validation:", cv_results['test_accuracy'].std())
print("Mean precision score from cross-validation:", cv_results['test_precision'].mean())
print("Standard deviation of precision score from cross-validation:", cv_results['test_precision'].std())


ValueError: ignored

In [ ]:
# predict on the training set and compute the accuracy and precision scores
y_train_pred = hist_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='weighted')

# predict on the validation set and compute the accuracy and precision scores
y_val_pred = hist_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_precision = precision_score(y_val, y_val_pred, average='weighted')

# calculate the percentage of positive predictions
train_pct_pos = sum(y_train_pred == 2) / len(y_train_pred) * 100
val_pct_pos = sum(y_val_pred == 2) / len(y_val_pred) * 100

# print the results
print("Training set accuracy:", train_accuracy)
print("Training set precision:", train_precision)
print("Validation set accuracy:", val_accuracy)
print("Validation set precision:", val_precision)
print("Percentage of positive predictions on training set:", train_pct_pos)
print("Percentage of positive predictions on validation set:", val_pct_pos)
print("Mean accuracy score from cross-validation:", cv_results['test_accuracy'].mean())
print("Standard deviation of accuracy score from cross-validation:", cv_results['test_accuracy'].std())
print("Mean precision score from cross-validation:", cv_results['test_precision'].mean())
print("Standard deviation of precision score from cross-validation:", cv_results['test_precision'].std())

Training set accuracy: 0.9422633969118983
Training set precision: 0.942213545010241
Validation set accuracy: 0.8283633060853769
Validation set precision: 0.8255465677788764
Percentage of positive predictions on training set: 31.414168937329702
Percentage of positive predictions on validation set: 28.88410535876476
Mean accuracy score from cross-validation: 0.8153605812897367
Standard deviation of accuracy score from cross-validation: 0.013187758997051281
Mean precision score from cross-validation: 0.81345279934022
Standard deviation of precision score from cross-validation: 0.01332682788402306


In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
hist_model = HistGradientBoostingClassifier(max_depth=3, max_iter=100, learning_rate=0.1)
hist_model.fit(X_train, y_train)

HistGradientBoostingClassifier(max_depth=3)

In [ ]:
from sklearn.model_selection import cross_val_score

hist_model = HistGradientBoostingClassifier(max_depth=3, max_iter=100, learning_rate=0.1)
scores = cross_val_score(hist_model, X, y_encoded, cv=5, scoring='accuracy')
print('Mean cross-validation score:', scores.mean())


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
# define the parameter grid
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.001, 0.01, 0.1, 1, 10],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 500, 1000]
}

# define the model
lr = LogisticRegression()

# define the scoring metrics
scoring = {'accuracy': 'accuracy', 'precision': make_scorer(precision_score, average='weighted')}
refit_score = 'precision'

# define the GridSearchCV object
grid_search = GridSearchCV(
    estimator=lr,
    param_grid=param_grid,
    scoring=scoring,
    refit=refit_score,
    cv=5
)

# fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# print the results
print(grid_search.cv_results_)


In [ ]:
# y_pred = hist_model.predict(X_val)
y_pred = hist_model.predict(X_val)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score


precision = precision_score(y_val, y_pred, average='weighted')
print("Precision:", precision)


y_pred = hist_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)


Precision: 0.789787157559818
Accuracy: 0.7934241598546775
